# Dynamic Binding, Dynamic Typing, Abstraction, Encapsulation —


---

# 1. Dynamic Binding

## Definition

Dynamic binding (also called late binding) is the runtime association of a name or operation with the implementation that will be executed. In object-oriented code, dynamic binding typically means the selection of which method implementation to call is determined at runtime based on the actual class of the object.

In Python this is the default: method calls and attribute accesses are resolved at runtime via the object’s class and the method resolution order.

## Variants / Related concepts

* Method dynamic binding (method dispatch): which method implementation an `obj.method()` call invokes.
* Name binding: mapping of names to objects (variables to objects) happens at runtime.
* Late binding in closures: a common pitfall where lambdas or inner functions capture variables by name and resolve them when invoked, not when created.
* Dynamic dispatch vs static dispatch: Python uses dynamic dispatch.

## Example code

### Method dynamic binding

```python
class A:
    def greet(self):
        return "A"

class B(A):
    def greet(self):
        return "B"

def call_greet(x):
    return x.greet()

a = A()
b = B()
print(call_greet(a))  # "A"
print(call_greet(b))  # "B"  (bound at runtime to B.greet)
```

### Late binding in closures (common pitfall)

```python
funcs = []
for i in range(3):
    funcs.append(lambda: i)   # captures name 'i', not its value

print([f() for f in funcs])  # [2, 2, 2] in Python — resolved when called
# correct approach: bind current value as default arg
funcs = []
for i in range(3):
    funcs.append(lambda i=i: i)
print([f() for f in funcs])  # [0, 1, 2]
```

## Internal runtime / memory behaviour (what happens)

* Attribute lookup for `obj.method()`:

  1. Python evaluates `obj` to an object reference (a pointer to a heap object).
  2. Calls `obj.__getattribute__('method')`.
  3. `__getattribute__` looks in the instance `__dict__` for the name; if not there, it looks in the class `__dict__` and then in base classes following MRO.
  4. If the attribute is a function object defined in the class, Python will create a *bound method* object by wrapping the function with the instance; this bound method holds a reference to the function object and the instance (so `bound_method.__self__` is the instance, `bound_method.__func__` is the function).
  5. That bound method object is called.
* Because lookup happens at call time, if a subclass has overridden the function name, the subclass function will be found first and used.
* The MRO (method resolution order) is computed when the class is created (C3 linearization). That MRO is a small array of class pointers used during attribute lookup.

## Pros

* Flexibility: you can substitute objects with compatible interfaces and behavior changes automatically.
* Extensibility: classes can override methods and new classes integrate without changing calling code.
* Dynamic behaviors: monkey patching, runtime replacement of functions/methods is possible.

## Cons

* Harder to reason statically: the concrete method executed is not known until runtime.
* Potential performance cost: attribute lookup and bound method creation occurs at call time (but CPython caches some lookups at the bytecode level and method calls are optimized).
* Can make debugging harder when methods are monkey-patched or when closures capture late-bound variables unintentionally.

## Best practices / when to use

* Use dynamic binding to implement polymorphism and plugins.
* Avoid heavy reliance on runtime mutation of class behaviour in production (monkey patching) — it complicates reasoning and testing.
* For closures where you need the current loop value, bind it as a default parameter to avoid late-binding surprises.

---

# 2. Dynamic Typing

## Definition

Dynamic typing means variable names are bound to objects and those objects carry their types at runtime; variables themselves are not statically typed. In Python, the type of an object is available at runtime (`type(obj)`), and operations are resolved based on the object’s runtime type.

Dynamic typing is distinct from duck typing (which is about interface, not explicit type).

## Variants / related ideas

* Duck typing: "if it quacks like a duck" — code depends on presence of methods/attributes rather than formal type.
* Structural typing (Protocols in `typing` module): a static type-checking approximation for dynamic languages.
* Gradual typing / optional static typing: `typing` module and type checkers (mypy, pyright) let you annotate and check types statically without changing runtime behaviour.

## Example code

### Dynamic typing and duck typing

```python
class Duck:
    def quack(self):
        return "quack"

class Person:
    def quack(self):
        return "I can imitate a duck"

def make_it_quack(x):
    # no type annotation required — we just call quack()
    return x.quack()

print(make_it_quack(Duck()))
print(make_it_quack(Person()))
```

### Using type hints (optional) + mypy

```python
from typing import Protocol

class Quacker(Protocol):
    def quack(self) -> str: ...

def make_it_quack(x: Quacker) -> str:
    return x.quack()
```

Type checkers can validate at development time; runtime behaviour is unchanged.

## Internal runtime / memory behaviour

* Names in Python (identifiers) are references to objects. A variable name is a slot in a namespace (local frame, cell, global dict, or attribute) that points to a PyObject* (in CPython).
* Objects themselves have type information: a pointer to a type object (`PyObject->ob_type` in CPython).
* Operations like attribute lookup, method resolution, operator semantics are all dispatched using the object's type object: e.g., `a + b` leads to a call to `a.__add__` or the type’s nb_add slot at the C API level.
* Rebinding a name simply replaces the pointer; there is no implicit data copy.

## Pros

* Rapid development and prototyping: fewer declarations.
* Very flexible: polymorphism, duck typing make APIs light-weight.
* Interactivity and REPL-friendly.

## Cons

* Runtime errors: type-related bugs that static languages would catch only surface at runtime.
* Harder to refactor without tooling: changing an API may break callers in non-obvious ways.
* Large codebases benefit from optional static checks; dynamic typing alone can be brittle.

## Alternatives and safety tools

* Use type hints (`typing`) and static type checkers (`mypy`, `pyright`) to get many benefits of static typing without changing runtime.
* Use unit tests to catch runtime type errors.
* Where performance matters, specialized types or C extensions can be used.

## Best practices

* Add meaningful type hints for public functions and complex modules.
* Use `isinstance()` guards where runtime type discrimination is required.
* Prefer duck typing for APIs where behavior (methods present) matters more than exact type, but document required methods clearly.

---

# 3. Abstraction

## Definition

Abstraction is the concept of exposing only the necessary features of an object or module while hiding implementation details. In OOP, abstraction typically appears as abstract base classes (ABCs) or interfaces that define required behaviors without dictating implementation.

In Python, abstraction is achieved via:

* Abstract Base Classes (`abc.ABC`, `@abstractmethod`)
* Protocols (`typing.Protocol`) for structural typing
* Designing public APIs and hiding internal parts via naming conventions and modules

## Types / forms

* Interface abstraction: ABCs and Protocols define method signatures clients rely upon.
* Data abstraction: exposing getters/setters or properties rather than internal representation.
* Module-level abstraction: `__all__` controls exported names, hide helper functions with leading underscore.

## Example code

### Abstract Base Class (abc)

```python
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

    @abstractmethod
    def perimeter(self):
        pass

class Circle(Shape):
    def __init__(self, r):
        self.r = r
    def area(self):
        import math
        return math.pi * self.r * self.r
    def perimeter(self):
        import math
        return 2 * math.pi * self.r

# Shape() cannot be instantiated; Circle must implement abstract methods
c = Circle(1.0)
print(c.area(), c.perimeter())
```

### Protocol (structural typing, works with static type checkers)

```python
from typing import Protocol

class Quacker(Protocol):
    def quack(self) -> str: ...

def make_it_quack(q: Quacker) -> str:
    return q.quack()
```

At runtime protocols are not enforced (unless `runtime_checkable` is used), but static tools can check.

## Internal runtime / memory behaviour

* ABC: when a class inherits from an ABC with abstract methods, the class object has a flag tracking abstract methods (stored in `__abstractmethods__`). Attempting to instantiate a class with unresolved abstracts raises `TypeError`.
* `@abstractmethod` marks function objects and the metaclass `ABCMeta` accumulates them; class creation populates `__abstractmethods__`.
* Protocols are primarily a static typing construct; they do not change the runtime inheritance chain unless `runtime_checkable` is used, which then allows `isinstance(obj, Protocol)` at runtime but still relies on duck typing.

## Pros

* Clear contracts: consumers of an API know what methods they can call.
* Decoupling: implementations can change without changing consumers as long as the interface is honored.
* Helps large team design and reasoning.

## Cons

* Over-abstraction: too many tiny interfaces can complicate design.
* Runtime enforcement in Python is limited—ABCs provide some runtime checks, but Protocols are mostly compile-time/static analysis aids.
* Can encourage rigid designs if used prematurely.

## Best practices

* Use ABCs when you need runtime checks (e.g., disallow instantiation of incomplete classes), or when providing a canonical abstract base in a library.
* Use Protocols and type hints when you want flexible, static-interface checks without enforcing inheritance.
* Keep abstractions coarse-grained and meaningful to the domain.

---

# 4. Encapsulation

## Definition

Encapsulation is the bundling of data and methods that operate on the data into a single unit (class), and restricting direct access to some of an object's components. It is a mechanism to hide internal state and require all interaction to be performed through an object's methods.

In Python, encapsulation is supported by conventions and language features:

* Public (no underscore)
* "Protected" (`_name`) — convention only
* "Private" (`__name`) — name mangling (not absolute privacy)
* Properties/descriptors (`@property`, descriptors) to control attribute access
* `__slots__` to restrict instance attribute creation (and reduce memory)

## Types / techniques

* Naming conventions: `_attr` and `__attr` (name mangled).
* Access control via `property` for read-only or computed attributes.
* Descriptors for custom attribute access behavior (implementing `__get__`, `__set__`, `__delete__`).
* `__slots__` to restrict attributes and improve memory usage.
* Module-level encapsulation: export control with `__all__`.

## Example code

### Basic private/protected usage

```python
class Person:
    def __init__(self, name, salary):
        self.name = name         # public
        self._salary = salary    # protected by convention
        self.__ssn = "secret"    # name-mangled private

    def get_ssn(self):
        return self.__ssn

p = Person("Alice", 1000)
print(p.name)
print(p._salary)            # discouraged but possible
# print(p.__ssn)           # AttributeError
print(p._Person__ssn)       # access via name-mangling
```

### Encapsulation via property

```python
class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Below absolute zero")
        self._celsius = value

    @property
    def fahrenheit(self):
        return self._celsius * 9 / 5 + 32

t = Temperature(25)
print(t.celsius, t.fahrenheit)
t.celsius = 100
# t.celsius = -300  -> raises ValueError
```

### Descriptor example (custom attribute control)

```python
class NonNegative:
    def __init__(self, name):
        self.name = "_" + name
    def __get__(self, instance, owner):
        return getattr(instance, self.name)
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Negative value not allowed")
        setattr(instance, self.name, value)

class Account:
    balance = NonNegative("balance")
    def __init__(self, balance):
        self.balance = balance

a = Account(100)
# a.balance = -10  -> raises ValueError
```

### **slots** to restrict attributes and reduce memory

```python
class Point:
    __slots__ = ("x", "y")  # instances only have x and y, no __dict__
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(1,2)
# p.z = 3  -> AttributeError
```

## Internal runtime / memory behaviour

* Name mangling (`__name`) transforms the attribute name at class creation time into `_ClassName__name` and stores that in the class dictionary. The instance then stores the attribute under that mangled name in its `__dict__`.
* `property` is a descriptor: the decorator wraps functions into a descriptor object that implements `__get__` and `__set__`. Attribute access triggers descriptor protocol and calls those methods instead of direct `__dict__` read/write.
* Descriptors are looked up on the class, not the instance; if a descriptor implements `__set__`, it is a data descriptor and takes precedence over instance attributes.
* `__slots__` replaces the per-instance `__dict__` with a fixed set of attribute slots; this reduces memory per instance and prevents creation of new arbitrary attributes but prohibits some dynamic behavior.
* Encapsulation is primarily by convention. The runtime still allows access to most internals (except that name mangling obscures but does not prevent access).

## Pros

* Prevents accidental external modification of internal state.
* Provides hooks for validation and computed attributes (`@property`).
* Makes class invariants easier to maintain and debug.
* `__slots__` can significantly reduce memory usage for large numbers of instances.

## Cons

* Python’s encapsulation is not absolute; determined code can access internals (via name mangling).
* Overuse of private members complicates subclassing and testing.
* `__slots__` limits flexibility (no `__dict__`, harder to add attributes at runtime; common pitfalls with multiple inheritance and slots).

## Best practices

* Use single underscore `_name` to signal internal use.
* Use `@property` for controlled access to fields rather than insisting on direct private fields.
* Use double underscore `__name` only when you need to avoid name collision in subclasses; prefer composition if heavy protection is required.
* Use `__all__` in modules to control public API; hide internals with `_` prefixed names.
* Use `__slots__` when you need memory optimization and your class structure is stable.

---

# Cross-topic notes and common internals (attribute lookup & method call lifecycle)

1. **Namespaces and frames**

   * Local variables live on the function frame (fast local array) or in closures (cells).
   * Globals are stored in the module dictionary.
   * Attribute lookups on objects use `__getattribute__` then `__getattr__` fallback.

2. **Attribute lookup order**

   * For `obj.attr`:

     * If `attr` is found in the instance `__dict__` and it's not overridden by a data descriptor on the class, return it.
     * Else, look at the class `__dict__` and then base classes following the MRO; if the found attribute is a descriptor, follow descriptor protocol.
     * If not found, raise `AttributeError` or fallback to `__getattr__`.

3. **Bound method object**

   * When you access a function from a class on an instance, Python creates a bound method object that pairs the function and the instance. That object carries references that keep the instance alive for the call.

4. **MRO (Method Resolution Order)**

   * MRO is computed at class creation time using C3 linearization. It is used to find methods in multiple inheritance scenarios and to ensure a consistent and monotonic order.

5. **Performance**

   * Attribute lookup and method calls are optimized in CPython, but repeated lookups inside tight loops are best avoided by local binding:

     ```python
     # slower
     for _ in range(1000000):
         obj.method()
     # faster
     m = obj.method
     for _ in range(1000000):
         m()
     ```
   * Bound method lookup costs are avoided by binding to a local variable.

---

# Practical recommendations

* For library/public APIs: use type hints for clarity and to facilitate static checking.
* For designing class hierarchies: prefer composition over deep inheritance chains.
* Use ABCs when you need runtime enforcement and Protocols when you want static structural checks.
* Use `@property` and descriptors to enforce invariants and perform validation; avoid mixing direct attribute writes with properties unexpectedly.
* Be explicit about intended encapsulation; prefer single underscore `_internal` and document it.

---



